<a href="https://colab.research.google.com/github/abirahsan007/PatternLab/blob/main/PatLab03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd /content/gdrive/My Drive/pattern lab files/Dataset

In [ ]:
import numpy as np
import pickle
import random

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, matthews_corrcoef
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [ ]:
Xtrain = np.load(f'./final datas/Xtrain.npy')
Ytrain = np.load(f'./final datas/Ytrain.npy')
Xtest = np.load(f'./final datas/Xtest.npy')
Ytest = np.load(f'./final datas/Ytest.npy')

with open('index_dict.pickle', 'rb') as handle:
    index_dict = pickle.load(handle)
index_dict

In [ ]:
random.seed(13)

features = {1 : '(CKSNAP1)', 2 : '(CKSNAP3)', 3 : '(CKSNAP5)', 4 : '(CKSNAP7)', 5 : '(CKSNAP9)', 6 : '(DAC7)', 7 : '(EIIP)', 8 : '(ENAC10)', 9 : '(ENAC5)', 10 : '(NCP)', 11 : '(PseEIIP)', 12 : '(TAC7)', 13 : '(binary)', 14 : '(kmer1)', 15 : '(kmer2)', 16 : '(kmer3)', 17 : '(kmer4)', 18 : '(kmer5)'}

subgrp1 = [features[random.randint(1,18)] for i in range(5)]
subgrp2 = [features[random.randint(1,18)] for i in range(5)]
subgrp3 = [features[random.randint(1,18)] for i in range(5)]
subgrp4 = [features[random.randint(1,18)] for i in range(5)]
subgrp5 = [features[random.randint(1,18)] for i in range(5)]


X1 = np.concatenate([Xtrain[ : ,  index_dict[x][0] : index_dict[x][1]+1] for x in subgrp1], axis = 1)
X2 = np.concatenate([Xtrain[ : ,  index_dict[x][0] : index_dict[x][1]+1] for x in subgrp2], axis = 1)
X3 = np.concatenate([Xtrain[ : ,  index_dict[x][0] : index_dict[x][1]+1] for x in subgrp3], axis = 1)
X4 = np.concatenate([Xtrain[ : ,  index_dict[x][0] : index_dict[x][1]+1] for x in subgrp4], axis = 1)
X5 = np.concatenate([Xtrain[ : ,  index_dict[x][0] : index_dict[x][1]+1] for x in subgrp5], axis = 1)

Xs = [X1, X2, X3, X4, X5]

In [ ]:
subgroups = [subgrp1, subgrp2, subgrp3, subgrp4, subgrp5]
for group in subgroups:
  print(group)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from numpy import mean

kf = KFold(n_splits=5, shuffle = True)

Logistic Regression

In [ ]:
lg = LogisticRegression(max_iter=500)

for i,X in enumerate(Xs):
  train_acc = []
  test_acc = []
  train_sensitivity = []
  test_sensitivity = []
  train_specificity = []
  test_specificity = []
  train_mcc = []
  test_mcc = []
  for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Ytrain[train_index], Ytrain[test_index]
    lg.fit(X_train, y_train)
    y_pred = lg.predict(X_train)
    train_acc.append(accuracy_score(y_train, y_pred))
    cf = confusion_matrix(y_train, y_pred)
    train_sensitivity.append(cf[0,0]/(cf[0,0] + cf[0,1]))
    train_specificity.append(cf[1,1]/(cf[1,0] + cf[1,1]))
    train_mcc.append(matthews_corrcoef(y_train, y_pred))

    lg.fit(X_train, y_train)
    y_pred = lg.predict(X_test)
    test_acc.append(accuracy_score(y_test, y_pred))
    cf = confusion_matrix(y_test, y_pred)
    test_sensitivity.append(cf[0,0]/(cf[0,0] + cf[0,1]))
    test_specificity.append(cf[1,1]/(cf[1,0] + cf[1,1]))
    test_mcc.append(matthews_corrcoef(y_test, y_pred))
  print(f'subgroup{i+1} accuracy in logistic regression \n train -> {mean(train_acc)}  test -> {mean(test_acc)}')
  print(f'subgroup{i+1} sensitivity in logistic regression \n train -> {mean(train_sensitivity)}  test -> {mean(test_sensitivity)}')
  print(f'subgroup{i+1} specificity in logistic regression \n train -> {mean(train_specificity)}  test -> {mean(test_specificity)}')
  print(f'subgroup{i+1} mcc in logistic regression \n train -> {mean(train_mcc)}  test -> {mean(test_mcc)}')

XG Boost

In [ ]:
xgbst = XGBClassifier()

for i,X in enumerate(Xs):
  train_acc = []
  test_acc = []
  train_sensitivity = []
  test_sensitivity = []
  train_specificity = []
  test_specificity = []
  train_mcc = []
  test_mcc = []
  for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Ytrain[train_index], Ytrain[test_index]
    xgbst.fit(X_train, y_train)
    y_pred = xgbst.predict(X_train)
    train_acc.append(accuracy_score(y_train, y_pred))
    cf = confusion_matrix(y_train, y_pred)
    train_sensitivity.append(cf[0,0]/(cf[0,0] + cf[0,1]))
    train_specificity.append(cf[1,1]/(cf[1,0] + cf[1,1]))
    train_mcc.append(matthews_corrcoef(y_train, y_pred))

    xgbst.fit(X_train, y_train)
    y_pred = xgbst.predict(X_test)
    test_acc.append(accuracy_score(y_test, y_pred))
    cf = confusion_matrix(y_test, y_pred)
    test_sensitivity.append(cf[0,0]/(cf[0,0] + cf[0,1]))
    test_specificity.append(cf[1,1]/(cf[1,0] + cf[1,1]))
    test_mcc.append(matthews_corrcoef(y_test, y_pred))
  print(f'subgroup{i+1} accuracy in XG boost \n train -> {mean(train_acc)}  test -> {mean(test_acc)}')
  print(f'subgroup{i+1} sensitivity in XG boost \n train -> {mean(train_sensitivity)}  test -> {mean(test_sensitivity)}')
  print(f'subgroup{i+1} specificity in XG boost \n train -> {mean(train_specificity)}  test -> {mean(test_specificity)}')
  print(f'subgroup{i+1} mcc in XG boost \n train -> {mean(train_mcc)}  test -> {mean(test_mcc)}')

Ada Boost

In [ ]:
ab = AdaBoostClassifier()

for i,X in enumerate(Xs):
  train_acc = []
  test_acc = []
  train_sensitivity = []
  test_sensitivity = []
  train_specificity = []
  test_specificity = []
  train_mcc = []
  test_mcc = []
  for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Ytrain[train_index], Ytrain[test_index]
    ab.fit(X_train, y_train)
    y_pred = ab.predict(X_train)
    train_acc.append(accuracy_score(y_train, y_pred))
    cf = confusion_matrix(y_train, y_pred)
    train_sensitivity.append(cf[0,0]/(cf[0,0] + cf[0,1]))
    train_specificity.append(cf[1,1]/(cf[1,0] + cf[1,1]))
    train_mcc.append(matthews_corrcoef(y_train, y_pred))

    ab.fit(X_train, y_train)
    y_pred = ab.predict(X_test)
    test_acc.append(accuracy_score(y_test, y_pred))
    cf = confusion_matrix(y_test, y_pred)
    test_sensitivity.append(cf[0,0]/(cf[0,0] + cf[0,1]))
    test_specificity.append(cf[1,1]/(cf[1,0] + cf[1,1]))
    test_mcc.append(matthews_corrcoef(y_test, y_pred))
  print(f'subgroup{i+1} accuracy in Ada boost \n train -> {mean(train_acc)}  test -> {mean(test_acc)}')
  print(f'subgroup{i+1} sensitivity in Ada boost \n train -> {mean(train_sensitivity)}  test -> {mean(test_sensitivity)}')
  print(f'subgroup{i+1} specificity in Ada boost \n train -> {mean(train_specificity)}  test -> {mean(test_specificity)}')
  print(f'subgroup{i+1} mcc in Ada boost \n train -> {mean(train_mcc)}  test -> {mean(test_mcc)}')

Random Forest

In [ ]:
rf = RandomForestClassifier()

for i,X in enumerate(Xs):
  train_acc = []
  test_acc = []
  train_sensitivity = []
  test_sensitivity = []
  train_specificity = []
  test_specificity = []
  train_mcc = []
  test_mcc = []
  for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Ytrain[train_index], Ytrain[test_index]
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_train)
    train_acc.append(accuracy_score(y_train, y_pred))
    cf = confusion_matrix(y_train, y_pred)
    train_sensitivity.append(cf[0,0]/(cf[0,0] + cf[0,1]))
    train_specificity.append(cf[1,1]/(cf[1,0] + cf[1,1]))
    train_mcc.append(matthews_corrcoef(y_train, y_pred))

    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    test_acc.append(accuracy_score(y_test, y_pred))
    cf = confusion_matrix(y_test, y_pred)
    test_sensitivity.append(cf[0,0]/(cf[0,0] + cf[0,1]))
    test_specificity.append(cf[1,1]/(cf[1,0] + cf[1,1]))
    test_mcc.append(matthews_corrcoef(y_test, y_pred))
  print(f'subgroup{i+1} accuracy in Random forest \n train -> {mean(train_acc)}  test -> {mean(test_acc)}')
  print(f'subgroup{i+1} sensitivity in Random forest \n train -> {mean(train_sensitivity)}  test -> {mean(test_sensitivity)}')
  print(f'subgroup{i+1} specificity in Random forest \n train -> {mean(train_specificity)}  test -> {mean(test_specificity)}')
  print(f'subgroup{i+1} mcc in Random forest \n train -> {mean(train_mcc)}  test -> {mean(test_mcc)}')

SVC

In [ ]:
svc = SVC()

for i,X in enumerate(Xs):
  train_acc = []
  test_acc = []
  train_sensitivity = []
  test_sensitivity = []
  train_specificity = []
  test_specificity = []
  train_mcc = []
  test_mcc = []
  for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Ytrain[train_index], Ytrain[test_index]
    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_train)
    train_acc.append(accuracy_score(y_train, y_pred))
    cf = confusion_matrix(y_train, y_pred)
    train_sensitivity.append(cf[0,0]/(cf[0,0] + cf[0,1]))
    train_specificity.append(cf[1,1]/(cf[1,0] + cf[1,1]))
    train_mcc.append(matthews_corrcoef(y_train, y_pred))

    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_test)
    test_acc.append(accuracy_score(y_test, y_pred))
    cf = confusion_matrix(y_test, y_pred)
    test_sensitivity.append(cf[0,0]/(cf[0,0] + cf[0,1]))
    test_specificity.append(cf[1,1]/(cf[1,0] + cf[1,1]))
    test_mcc.append(matthews_corrcoef(y_test, y_pred))
  print(f'subgroup{i+1} accuracy in svc \n train -> {mean(train_acc)}  test -> {mean(test_acc)}')
  print(f'subgroup{i+1} sensitivity in svc \n train -> {mean(train_sensitivity)}  test -> {mean(test_sensitivity)}')
  print(f'subgroup{i+1} specificity in svc \n train -> {mean(train_specificity)}  test -> {mean(test_specificity)}')
  print(f'subgroup{i+1} mcc in svc \n train -> {mean(train_mcc)}  test -> {mean(test_mcc)}')

Independent Test(Logistic Regression)

In [ ]:
lg = LogisticRegression(max_iter=500)

acc = []
sensitivity = []
specificity = []
mcc = []
lg.fit(Xtrain, Ytrain)
y_pred = lg.predict(Xtest)
acc = accuracy_score(Ytest, y_pred)
cf = confusion_matrix(Ytest, y_pred)
sensitivity = (cf[0,0]/(cf[0,0] + cf[0,1]))
specificity = (cf[1,1]/(cf[1,0] + cf[1,1]))
mcc = matthews_corrcoef(Ytest, y_pred)

print(f'acc -> {acc} \n sensitivity -> {sensitivity} \n specificity -> {specificity} \n mcc -> {mcc}')

Independent Test(XG Boost)

In [ ]:
xgbst = XGBClassifier()

acc = []
sensitivity = []
specificity = []
mcc = []
xgbst.fit(Xtrain, Ytrain)
y_pred = xgbst.predict(Xtest)
acc = accuracy_score(Ytest, y_pred)
cf = confusion_matrix(Ytest, y_pred)
sensitivity = (cf[0,0]/(cf[0,0] + cf[0,1]))
specificity = (cf[1,1]/(cf[1,0] + cf[1,1]))
mcc = matthews_corrcoef(Ytest, y_pred)

print(f'acc -> {acc} \n sensitivity -> {sensitivity} \n specificity -> {specificity} \n mcc -> {mcc}')

Independent Test(Ada Boost)

In [ ]:
ab = AdaBoostClassifier()

acc = []
sensitivity = []
specificity = []
mcc = []
ab.fit(Xtrain, Ytrain)
y_pred = ab.predict(Xtest)
acc = accuracy_score(Ytest, y_pred)
cf = confusion_matrix(Ytest, y_pred)
sensitivity = (cf[0,0]/(cf[0,0] + cf[0,1]))
specificity = (cf[1,1]/(cf[1,0] + cf[1,1]))
mcc = matthews_corrcoef(Ytest, y_pred)

print(f'acc -> {acc} \n sensitivity -> {sensitivity} \n specificity -> {specificity} \n mcc -> {mcc}')

Independent Test(Rand. forest)

In [ ]:
rf = RandomForestClassifier()

acc = []
sensitivity = []
specificity = []
mcc = []
rf.fit(Xtrain, Ytrain)
y_pred = rf.predict(Xtest)
acc = accuracy_score(Ytest, y_pred)
cf = confusion_matrix(Ytest, y_pred)
sensitivity = (cf[0,0]/(cf[0,0] + cf[0,1]))
specificity = (cf[1,1]/(cf[1,0] + cf[1,1]))
mcc = matthews_corrcoef(Ytest, y_pred)

print(f'acc -> {acc} \n sensitivity -> {sensitivity} \n specificity -> {specificity} \n mcc -> {mcc}')

Independent Test(SVC)

In [ ]:
svc = SVC()

acc = []
sensitivity = []
specificity = []
mcc = []
svc.fit(Xtrain, Ytrain)
y_pred = svc.predict(Xtest)
acc = accuracy_score(Ytest, y_pred)
cf = confusion_matrix(Ytest, y_pred)
sensitivity = (cf[0,0]/(cf[0,0] + cf[0,1]))
specificity = (cf[1,1]/(cf[1,0] + cf[1,1]))
mcc = matthews_corrcoef(Ytest, y_pred)

print(f'acc -> {acc} \n sensitivity -> {sensitivity} \n specificity -> {specificity} \n mcc -> {mcc}')